In [23]:
import numpy as np
import pandas as pd

In [56]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [53]:
df = pd.read_csv('train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [26]:
df = df.drop(['PassengerId','Name','Ticket','Fare','Cabin'],axis=1)

In [27]:
df.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C


In [28]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Embarked      2
dtype: int64

In [43]:
# spliting the data
np.random.seed(42)
x = df.drop('Survived',axis=1)
y = df['Survived']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [129]:
x_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Embarked
331,1,male,45.5,0,0,S
733,2,male,23.0,0,0,S


In [86]:
# creating the transformer for imputation
trans1 = ColumnTransformer([
    ('imputer_age',SimpleImputer(),[2]),
    ('imputer_embrak',SimpleImputer(strategy='most_frequent'),[5])
],remainder='passthrough')

In [87]:
# creating the transformer for onehotencoder
trans2 = ColumnTransformer([
    ('ohe_sex_embrak',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,5])
],remainder='passthrough')

In [92]:
# Creating the trnsformer for feature scaling
trans3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
],)

In [93]:
# creating trans for algorithm
trans4= RandomForestClassifier()

In [94]:
# preparing the pipeline
pipe = Pipeline([
    ('trans1',trans1),
    ('trans2',trans2),
    ('trans3',trans3),
    ('trans4',trans4)
])

# Pipeline Vs make_pipeline
Pipeline requires naming of steps,where's make_pipeline does not.
(same applies to columnTransformer and make_columnTransformer)

In [63]:
pipe = make_pipeline(trans1,trans2,trans3,trans4)

In [95]:

pipe.fit(x_train,y_train)

Pipeline(steps=[('trans1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('imputer_age',
                                                  SimpleImputer(), [2]),
                                                 ('imputer_embrak',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [5])])),
                ('trans2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embrak',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 5])])),
                ('trans3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trans4', RandomForestClassifier())])

In [61]:
y_pred = pipe.predict(x_test)

In [62]:
print(accuracy_score(y_test,y_pred))

0.6368715083798883


## Explore the pipeline

In [105]:
pipe.named_steps['trans1'].transformers_[0][1].statistics_

array([29.49884615])

# corss validation

In [106]:
from sklearn.model_selection import cross_val_score


In [109]:
np.mean(cross_val_score(pipe,x,y,scoring='accuracy',cv=5))

0.6341974766179147

# Tuning using GridSearchCV
 

In [111]:
from sklearn.model_selection import GridSearchCV

In [120]:
params={
    'trans4__n_estimators':[100,400,200,500],
    'trans4__min_samples_split':[2,3,4],
    'trans4__criterion':["gini", "entropy", "log_loss"],
    'trans4__max_features':["sqrt", "log2"],
    "trans4__max_depth":[1,2,3,None]
}

In [121]:
grid_model = GridSearchCV(pipe,param_grid=params,cv=5,scoring='accuracy')
grid_model.fit(x_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trans1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('imputer_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('imputer_embrak',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [5])])),
                                       ('trans2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embrak',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1...
                                       ('trans3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trans4', RandomForestClassifier())]),
             param_grid={'trans4__criterion': ['gini', 'entropy', 'log_loss'],
                         'trans4__max_depth': [1, 2, 3, None],
                         'trans4__max_features': ['sqrt', 'log2'],
                         'trans4__min_samples_split': [2, 3, 4],
                         'trans4__n_estimators': [100, 400, 200, 500]},
             scoring='accuracy')

In [122]:
grid_model.best_score_

0.6432975475228997

# Exporting the Pipeline

In [123]:
# export
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))

In [124]:
pipe = pickle.load(open('pipe.pkl','rb'))

In [130]:
# Assume user input
test_input1 = np.array([ '1','male','45.5','0','0','S'],dtype=object).reshape(1,6)

In [131]:
pipe.predict(test_input1)

array([0], dtype=int64)